In [232]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import sklearn
from sklearn.model_selection import train_test_split
# import dask.dataframe as dd
from warnings import filterwarnings
filterwarnings('ignore')

### Loading the data

In [22]:

df_business = pd.DataFrame(pd.read_json(r'.//yelp_dataset//yelp_academic_dataset_business.json', lines=True))
print("Business DataFrame:", df_business.shape)

df_checkin = pd.DataFrame(pd.read_json(r'.//yelp_dataset//yelp_academic_dataset_checkin.json', lines=True))
print("Checkin DataFrame:", df_checkin.shape)

df_tip = pd.DataFrame(pd.read_json(r'.//yelp_dataset//yelp_academic_dataset_tip.json', lines=True))
print("Tip DataFrame:", df_tip.shape)

df_user = pd.DataFrame(pd.read_json(r'.//yelp_dataset//yelp_academic_dataset_user.json', lines=True))
print("User DataFrame:", df_user.shape)

size = 10000
review_file_path = r'.//yelp_dataset//yelp_academic_dataset_review.json'
review_chunks = pd.read_json(review_file_path, lines=True, chunksize=size)
df_review = pd.concat(review_chunks, ignore_index=True)
print("Review DataFrame:", df_review.shape)




Business DataFrame: (150346, 14)
Checkin DataFrame: (131930, 2)
Tip DataFrame: (908915, 5)
User DataFrame: (1987897, 22)
Review DataFrame: (6990280, 9)


### Preprocessing Review Table

In [23]:
df_review.head(1)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11


In [24]:
df_review.isnull().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

In [25]:
df_review.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

In [26]:
df_review.drop(columns=['text','date'],axis=1,inplace = True)
# dropping the actual review for now as attributes of the text have already been feature engineered
# dropping date
# dropping review ID as it is not present in any other feature and is just an ID

In [27]:
df_review[df_review.duplicated()] # dropping duplicates

,review_id,user_id,business_id,stars,useful,funny,cool


In [28]:
df_review.drop_duplicates(inplace = True)
df_review.reset_index(drop=True,inplace = True)

In [29]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 7 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   review_id    object
 1   user_id      object
 2   business_id  object
 3   stars        int64 
 4   useful       int64 
 5   funny        int64 
 6   cool         int64 
dtypes: int64(4), object(3)
memory usage: 373.3+ MB


In [30]:
# reducing the data type to save memory allocation

# Convert object columns to category columns
for col in df_review.select_dtypes(include='object').columns:
    df_review[col] = df_review[col].astype('category')


In [31]:
# Convert int64 columns to int lower
for col in df_review.select_dtypes(include='int64').columns:
    df_review[col] = pd.to_numeric(df_review[col], downcast='integer')


In [32]:
df_review.info()        # data types of the features have been reduced

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 7 columns):
 #   Column       Dtype   
---  ------       -----   
 0   review_id    category
 1   user_id      category
 2   business_id  category
 3   stars        int8    
 4   useful       int16   
 5   funny        int16   
 6   cool         int16   
dtypes: category(3), int16(3), int8(1)
memory usage: 522.8 MB


In [33]:
# df_review.to_csv('Yelp_Review.csv',index=False)

### Preprocessing User Table

In [34]:
df_user.head(1)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180


In [35]:
df_user[df_user.duplicated()] # no duplicates

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos


In [36]:
df_user.isnull().sum()

user_id               0
name                  0
review_count          0
yelping_since         0
useful                0
funny                 0
cool                  0
elite                 0
friends               0
fans                  0
average_stars         0
compliment_hot        0
compliment_more       0
compliment_profile    0
compliment_cute       0
compliment_list       0
compliment_note       0
compliment_plain      0
compliment_cool       0
compliment_funny      0
compliment_writer     0
compliment_photos     0
dtype: int64

In [38]:
df_user.columns

Index(['user_id', 'name', 'review_count', 'average_stars'], dtype='object')

In [37]:
df_user.drop(columns = ['yelping_since','friends','compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos','fans','elite','useful','funny','cool'],axis=1,inplace = True)

# Dropping yelping since as a user's general yelping activity won't be strongly correlated with the business being open or no
# dropping friends, fans, elite
# dropping the compliments the user has received as that won't be related to if a business is open or not.

In [39]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   user_id        object 
 1   name           object 
 2   review_count   int64  
 3   average_stars  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 60.7+ MB


In [40]:
# reducing the data type to save memory allocation

# Convert object columns to category columns
for col in df_user.select_dtypes(include='object').columns:
    df_user[col] = df_user[col].astype('category')


In [41]:
# Convert int64 columns to int lower
for col in df_user.select_dtypes(include='int64').columns:
    df_user[col] = pd.to_numeric(df_user[col], downcast='integer')


In [42]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 4 columns):
 #   Column         Dtype   
---  ------         -----   
 0   user_id        category
 1   name           category
 2   review_count   int16   
 3   average_stars  float64 
dtypes: category(2), float64(1), int16(1)
memory usage: 118.8 MB


In [43]:
# df_user.to_csv('Yelp_Users.csv',index=False)

### Preprocessing Tip Table

In [44]:
df_tip.head(1)

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0


In [45]:
df_tip[df_tip.duplicated()] # dropping duplicates
df_tip.drop_duplicates(inplace = True)
df_tip.reset_index(drop=True,inplace = True)

In [46]:
df_tip.columns

Index(['user_id', 'business_id', 'text', 'date', 'compliment_count'], dtype='object')

In [47]:
df_tip.isnull().sum()

user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64

In [48]:
df_tip.drop(columns = ['text','date'],axis=1,inplace = True)

# dropping the actual review text and the date for inital model performance

In [49]:
df_tip[df_tip.duplicated()]
df_tip.drop_duplicates(inplace = True)
df_tip.reset_index(drop=True,inplace = True)

In [50]:
df_tip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787265 entries, 0 to 787264
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           787265 non-null  object
 1   business_id       787265 non-null  object
 2   compliment_count  787265 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 18.0+ MB


In [51]:
# reducing the data type to save memory allocation

# Convert object columns to category columns
for col in df_tip.select_dtypes(include='object').columns:
    df_tip[col] = df_tip[col].astype('category')

# Convert int64 columns to int lower
for col in df_tip.select_dtypes(include='int64').columns:
    df_tip[col] = pd.to_numeric(df_tip[col], downcast='integer')


In [52]:
df_tip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787265 entries, 0 to 787264
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   user_id           787265 non-null  category
 1   business_id       787265 non-null  category
 2   compliment_count  787265 non-null  int8    
dtypes: category(2), int8(1)
memory usage: 22.0 MB


In [53]:
# df_tip.to_csv('Yelp_Tips.csv',index=False)

### Preprocessing CheckIN Table

In [54]:
df_checkin.date[0]

# Assuming the date is the date when a review was given to that feature. We will be feature engineering the number of reviews

def review_count(dates):
    return len(dates.split(','))

df_checkin['Review_count'] = df_checkin.date.apply(review_count)

In [55]:
df_checkin[df_checkin.duplicated()] # no duplicates

,business_id,date,Review_count


In [56]:
df_checkin.head(1)

,business_id,date,Review_count
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020...",11


In [57]:
# Dropping the feature after feature engineering
df_checkin.drop(columns = ['date'],axis=1,inplace = True)

In [58]:
df_checkin[df_checkin.duplicated()] # no duplicates

,business_id,Review_count


In [59]:
df_checkin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   131930 non-null  object
 1   Review_count  131930 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.0+ MB


In [60]:
# reducing the data type to save memory allocation

# Convert object columns to category columns
for col in df_checkin.select_dtypes(include='object').columns:
    df_checkin[col] = df_checkin[col].astype('category')

# Convert int64 columns to int lower
for col in df_checkin.select_dtypes(include='int64').columns:
    df_checkin[col] = pd.to_numeric(df_checkin[col], downcast='integer')


In [61]:
df_checkin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   business_id   131930 non-null  category
 1   Review_count  131930 non-null  int32   
dtypes: category(1), int32(1)
memory usage: 6.0 MB


In [62]:
# df_checkin.to_csv('Yelp_CheckIns.csv',index=False)

### Preprocessing Business Table

In [63]:
df_business.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."


In [64]:
df_business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [65]:
df_business.isnull().sum() # dropping null rows

df_business.dropna(axis=0,inplace = True)
df_business.reset_index(drop=True,inplace=True)

#### Handling Attributes

In [66]:
df_business.attributes

0                    {'BusinessAcceptsCreditCards': 'True'}
1         {'BikeParking': 'True', 'BusinessAcceptsCredit...
2         {'RestaurantsDelivery': 'False', 'OutdoorSeati...
3         {'BusinessAcceptsCreditCards': 'True', 'Wheelc...
4         {'BusinessParking': 'None', 'BusinessAcceptsCr...
                                ...                        
117613    {'Ambience': '{'touristy': False, 'hipster': F...
117614    {'ByAppointmentOnly': 'False', 'RestaurantsPri...
117615    {'BusinessAcceptsCreditCards': 'True', 'Restau...
117616    {'BusinessParking': '{'garage': False, 'street...
117617    {'WheelchairAccessible': 'True', 'BusinessAcce...
Name: attributes, Length: 117618, dtype: object

In [67]:
df_temp = pd.json_normalize(df_business.attributes)
df_temp.head(3)

,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,CoatCheck,RestaurantsTakeOut,RestaurantsDelivery,Caters,WiFi,BusinessParking,WheelchairAccessible,...,AcceptsInsurance,BestNights,BYOB,Corkage,BYOBCorkage,HairSpecializesIn,Open24Hours,RestaurantsCounterService,AgesAllowed,DietaryRestrictions
0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,True,2,False,False,False,False,u'no',"{'garage': False, 'street': False, 'validated'...",True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,True,1,NaN,True,False,True,u'free',"{'garage': False, 'street': True, 'validated':...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
df_business = pd.concat([df_business,df_temp],axis=1)

In [69]:
df_business.drop('attributes',axis=1,inplace = True)

In [70]:
df_business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'categories', 'hours', 'BusinessAcceptsCreditCards', 'BikeParking',
       'RestaurantsPriceRange2', 'CoatCheck', 'RestaurantsTakeOut',
       'RestaurantsDelivery', 'Caters', 'WiFi', 'BusinessParking',
       'WheelchairAccessible', 'HappyHour', 'OutdoorSeating', 'HasTV',
       'RestaurantsReservations', 'DogsAllowed', 'ByAppointmentOnly',
       'Alcohol', 'GoodForKids', 'RestaurantsAttire', 'Ambience',
       'RestaurantsTableService', 'RestaurantsGoodForGroups', 'DriveThru',
       'GoodForMeal', 'NoiseLevel', 'BusinessAcceptsBitcoin', 'Smoking',
       'Music', 'GoodForDancing', 'AcceptsInsurance', 'BestNights', 'BYOB',
       'Corkage', 'BYOBCorkage', 'HairSpecializesIn', 'Open24Hours',
       'RestaurantsCounterService', 'AgesAllowed', 'DietaryRestrictions'],
      dtype='object')

In [112]:
# df_temp.to_csv('temp.csv',index=False)

In [117]:
# for col in df_temp.columns.tolist():
#     print(df_temp[col].unique())

In [81]:
null_data = pd.DataFrame((df_business.isnull().sum()*100)/df_business.shape[0],columns=['Null Percentage'])
null_data[null_data['Null Percentage']!=0].sort_values('Null Percentage',ascending = True)

,Null Percentage
BusinessAcceptsCreditCards,10.407421
BusinessParking,33.070618
RestaurantsPriceRange2,37.813090
BikeParking,44.481287
RestaurantsTakeOut,55.103811
WiFi,55.671751
RestaurantsDelivery,57.520958
GoodForKids,61.541601
OutdoorSeating,63.709636
HasTV,66.067269


In [169]:
# Assuming df is your dataframe
threshold = 50
selected_columns = df_business.columns[df_business.isnull().mean() * 100 <= threshold]
df_business_new = df_business[selected_columns]
df_business_new.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,BusinessParking
0,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",True,NaN,NaN,NaN


In [170]:
df_business_new.shape

(117618, 17)

In [171]:
df_business_new.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'categories', 'hours', 'BusinessAcceptsCreditCards', 'BikeParking',
       'RestaurantsPriceRange2', 'BusinessParking'],
      dtype='object')

In [172]:
df_business_new.BikeParking.value_counts()

BikeParking
True     50055
False    15181
None        64
Name: count, dtype: int64

In [173]:
df_business_new = df_business_new[df_business_new.BikeParking !='None']

In [174]:
# df_business_new[df_business_new.BikeParking.isnull()]


# Handle this later

In [175]:
df_business_new.BusinessAcceptsCreditCards.value_counts()

BusinessAcceptsCreditCards
True     100600
False      4667
None         58
Name: count, dtype: int64

In [176]:
df_business_new.BusinessAcceptsCreditCards.replace({'None':'False'},inplace = True)
# df_business_new.BusinessAcceptsCreditCards.fillna('False',inplace = True)

df_business_new.RestaurantsPriceRange2.replace({'None':0},inplace = True)
# df_business_new.RestaurantsPriceRange2.fillna(0,inplace = True)

df_business_new.BusinessParking.replace({'None':np.nan},inplace = True)
df_business_new.dropna(subset=['BusinessParking'],inplace =True)

df_business_new.reset_index(drop=True,inplace=True)

In [177]:
df_business_new.isnull().sum()

business_id                       0
name                              0
address                           0
city                              0
state                             0
postal_code                       0
latitude                          0
longitude                         0
stars                             0
review_count                      0
is_open                           0
categories                        0
hours                             0
BusinessAcceptsCreditCards        0
BikeParking                   15869
RestaurantsPriceRange2            0
BusinessParking                   0
dtype: int64

In [178]:
import ast

df_business_new['BusinessParking'] = [ast.literal_eval(item) for item in df_business_new['BusinessParking']]
df_business_new1 = pd.json_normalize(df_business_new['BusinessParking'])

df_business_new1 = pd.concat([df_business_new,df_business_new1],axis=1)
df_business_new1.drop('BusinessParking',axis=1,inplace = True)

In [188]:
df_business_new1.isnull().sum()

business_id                   0
name                          0
address                       0
city                          0
state                         0
postal_code                   0
latitude                      0
longitude                     0
stars                         0
review_count                  0
is_open                       0
categories                    0
hours                         0
BusinessAcceptsCreditCards    0
BikeParking                   0
RestaurantsPriceRange2        0
garage                        0
street                        0
validated                     0
lot                           0
valet                         0
dtype: int64

In [180]:
# df_business_new1.garage.fillna(False,inplace = True)
# df_business_new1.street.fillna(False,inplace = True)
# df_business_new1.validated.fillna(False,inplace = True)
# df_business_new1.lot.fillna(True,inplace = True)
# df_business_new1.valet.fillna(False,inplace = True)


In [181]:
df_business_new1.dropna(subset=['garage'],inplace = True)
df_business_new1.dropna(subset=['street'],inplace = True)
df_business_new1.dropna(subset=['validated'],inplace = True)
df_business_new1.dropna(subset=['lot'],inplace = True)
df_business_new1.dropna(subset=['valet'],inplace = True)
df_business_new1.dropna(subset=['BikeParking'],inplace = True)

In [203]:
df_business_new1.validated.value_counts()

validated
0    75813
1      811
Name: count, dtype: int64

In [184]:
# df_business_new1.BikeParking.fillna('True',inplace = True)

In [187]:
df_business_new1.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,categories,hours,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,garage,street,validated,lot,valet
0,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,...,"Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",True,True,2,False,False,False,True,False
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",False,True,1,False,True,False,False,False


In [189]:
df_business_new1.BusinessAcceptsCreditCards.replace({'False':0,'True':1},inplace = True)
df_business_new1.garage.replace({False:int(0),True:int(1)},inplace = True)
df_business_new1.street.replace({False:0,True:1},inplace = True)
df_business_new1.validated.replace({False:0,True:1},inplace = True)
df_business_new1.lot.replace({False:0,True:1},inplace = True)
df_business_new1.valet.replace({False:0,True:1},inplace = True)



In [190]:
df_business_new1.isnull().sum()

business_id                   0
name                          0
address                       0
city                          0
state                         0
postal_code                   0
latitude                      0
longitude                     0
stars                         0
review_count                  0
is_open                       0
categories                    0
hours                         0
BusinessAcceptsCreditCards    0
BikeParking                   0
RestaurantsPriceRange2        0
garage                        0
street                        0
validated                     0
lot                           0
valet                         0
dtype: int64

In [194]:
df_business_new1.drop(columns = ['name','address','postal_code','latitude', 'longitude','hours'],axis=1,inplace = True)

# name of store is not necessary when we have ID. Similarly for address
# Dropping postal code, latitude and longitude. Keeping city and state. As these 5 features will mostly have very high multicollinearity between them which we don't want.
# Dropping hours when the business is open or not as well

In [195]:
df_business_new1[df_business_new1.duplicated()]    # no duplicates

,business_id,city,state,stars,review_count,is_open,categories,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,garage,street,validated,lot,valet


In [196]:
df_business_new1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76624 entries, 0 to 76623
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   business_id                 76624 non-null  object 
 1   city                        76624 non-null  object 
 2   state                       76624 non-null  object 
 3   stars                       76624 non-null  float64
 4   review_count                76624 non-null  int64  
 5   is_open                     76624 non-null  int64  
 6   categories                  76624 non-null  object 
 7   BusinessAcceptsCreditCards  76624 non-null  int64  
 8   BikeParking                 76624 non-null  object 
 9   RestaurantsPriceRange2      76624 non-null  object 
 10  garage                      76624 non-null  int64  
 11  street                      76624 non-null  int64  
 12  validated                   76624 non-null  int64  
 13  lot                         766

In [197]:
# reducing the data type to save memory allocation

# Convert object columns to category columns
for col in df_business_new1.select_dtypes(include='object').columns:
    df_business_new1[col] = df_business_new1[col].astype('category')

# Convert int64 columns to int lower
for col in df_business_new1.select_dtypes(include='int64').columns:
    df_business_new1[col] = pd.to_numeric(df_business_new1[col], downcast='integer')


In [198]:
df_business_new1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76624 entries, 0 to 76623
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   business_id                 76624 non-null  category
 1   city                        76624 non-null  category
 2   state                       76624 non-null  category
 3   stars                       76624 non-null  float64 
 4   review_count                76624 non-null  int16   
 5   is_open                     76624 non-null  int8    
 6   categories                  76624 non-null  category
 7   BusinessAcceptsCreditCards  76624 non-null  int8    
 8   BikeParking                 76624 non-null  category
 9   RestaurantsPriceRange2      76624 non-null  category
 10  garage                      76624 non-null  int8    
 11  street                      76624 non-null  int8    
 12  validated                   76624 non-null  int8    
 13  lot             

In [199]:
df_business_new1.head()

,business_id,city,state,stars,review_count,is_open,categories,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,garage,street,validated,lot,valet
0,tUFrWirKiKi_TAnsVWINQQ,Tucson,AZ,3.5,22,0,"Department Stores, Shopping, Fashion, Home & G...",1,True,2,0,0,0,1,0
1,MTSW4McQd7CbVtyjqoe9mw,Philadelphia,PA,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",0,True,1,0,1,0,0,0
2,mWMc6_wTdE0EUBKIGXDVfA,Green Lane,PA,4.5,13,1,"Brewpubs, Breweries, Food",1,True,0,0,0,0,1,0
3,n_0UpQx1hsNbnPUSlodU8w,Brentwood,MO,2.5,13,1,"Sporting Goods, Fashion, Shoe Stores, Shopping...",1,True,2,0,0,0,1,0
4,bBDDEgkFA1Otx9Lfe7BZUQ,Nashville,TN,1.5,10,1,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",1,True,1,0,0,0,0,0


In [200]:
# df_business.to_csv('Yelp_Business.csv',index=False)

In [201]:
# df_business_new1.to_csv('Yelp_Business_New.csv',index=False)

## TARGET FEATURE

In [290]:
df_business_new1['is_open'].value_counts()

is_open
1    53569
0    18352
Name: count, dtype: int64

In [202]:
print("Business DataFrame:", df_business_new1.shape)
print("Review DataFrame:", df_review.shape)
print("User DataFrame:", df_user.shape)
print("Tip DataFrame:", df_tip.shape)
print("Checkin DataFrame:", df_checkin.shape)


Business DataFrame: (76624, 15)
Review DataFrame: (6990280, 7)
User DataFrame: (1987897, 4)
Tip DataFrame: (787265, 3)
Checkin DataFrame: (131930, 2)


### Merging data

Done on DB browser - mergins_dfs.sql

In [204]:
df_final = pd.read_csv('merged_yelp_data_A3.csv')
print(df_final.shape)

(5580595, 26)


In [205]:
df_final.isnull().sum()

business_id                         0
city                                0
state                               0
business_stars                      0
business_review_count               0
is_open                             0
categories                          0
BusinessAcceptsCreditCards          0
restaurant_price_level              0
BikeParking                         0
parking_garage                      0
parking_street                      0
validated_parking                   0
lot_parking                         0
valet_parking                       0
review_id                           0
review_stars                        0
review_useful                       0
review_funny                        0
review_cool                         0
user_id                             9
user_name                          91
user_review_count                   9
user_average_stars                  9
tip_compliment_count          5303415
checkin_review_count            18394
dtype: int64

dropping tip_compliment_count

In [206]:
percentage_null = (100 * df_final['tip_compliment_count'].isnull().sum()) / df_final.shape[0]

print(percentage_null)      # dropping this feature as it has 95% null values
# df_final.drop('tip_compliment_count',axis=1,inplace = True)
df_final.drop('tip_compliment_count',axis=1,inplace = True)

95.03314610718033


In [210]:
df_final = df_final.dropna(subset=['user_id','user_review_count','user_average_stars'],how = 'all')
df_final.reset_index(drop=True,inplace = True)
print(df_final.shape)

(5580586, 25)


In [213]:
df_final.drop('user_name',axis=1,inplace = True)

In [215]:
df_final.reset_index(drop=True,inplace=True)
df_final.shape

(5580586, 24)

In [226]:
df_final.isnull().sum()

business_id                       0
city                              0
state                             0
business_stars                    0
business_review_count             0
is_open                           0
categories                        0
BusinessAcceptsCreditCards        0
restaurant_price_level            0
BikeParking                       0
parking_garage                    0
parking_street                    0
validated_parking                 0
lot_parking                       0
valet_parking                     0
review_id                         0
review_stars                      0
review_useful                     0
review_funny                      0
review_cool                       0
user_id                           0
user_review_count                 0
user_average_stars                0
checkin_review_count          18394
dtype: int64

### Handling Missing Values after train test split to prevent data leakage

In [224]:
df_final = df_final.drop_duplicates(subset=['business_id', 'review_id', 'user_id'])

In [225]:
df_final.shape

(5578603, 24)

In [233]:
df_final.columns

Index(['business_id', 'city', 'state', 'business_stars',
       'business_review_count', 'is_open', 'categories',
       'BusinessAcceptsCreditCards', 'restaurant_price_level', 'BikeParking',
       'parking_garage', 'parking_street', 'validated_parking', 'lot_parking',
       'valet_parking', 'review_id', 'review_stars', 'review_useful',
       'review_funny', 'review_cool', 'user_id', 'user_review_count',
       'user_average_stars', 'checkin_review_count'],
      dtype='object')

In [235]:
X = df_final.drop(columns=['validated_parking'])
y = df_final['validated_parking']

# Perform train-test split (80% train, 20% test by default)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# Display the shapes of the resulting sets
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (4462882, 23)
X_test shape: (1115721, 23)
y_train shape: (4462882,)
y_test shape: (1115721,)


In [245]:
X_train.isnull().sum()

business_id                   0
city                          0
state                         0
business_stars                0
business_review_count         0
is_open                       0
categories                    0
BusinessAcceptsCreditCards    0
restaurant_price_level        0
BikeParking                   0
parking_garage                0
parking_street                0
lot_parking                   0
valet_parking                 0
review_id                     0
review_stars                  0
review_useful                 0
review_funny                  0
review_cool                   0
user_id                       0
user_review_count             0
user_average_stars            0
checkin_review_count          0
dtype: int64

In [238]:
X_train.checkin_review_count.describe()

count    4.448283e+06
mean     9.144177e+02
std      2.170389e+03
min      1.000000e+00
25%      9.400000e+01
50%      3.150000e+02
75%      8.940000e+02
max      4.010900e+04
Name: checkin_review_count, dtype: float64

The mean > median - positvely skewed. So, replace missing values with median

In [239]:
X_train.checkin_review_count.fillna(X_train.checkin_review_count.median(),inplace = True)

In [243]:
X_test.isnull().sum()

business_id                   0
city                          0
state                         0
business_stars                0
business_review_count         0
is_open                       0
categories                    0
BusinessAcceptsCreditCards    0
restaurant_price_level        0
BikeParking                   0
parking_garage                0
parking_street                0
lot_parking                   0
valet_parking                 0
review_id                     0
review_stars                  0
review_useful                 0
review_funny                  0
review_cool                   0
user_id                       0
user_review_count             0
user_average_stars            0
checkin_review_count          0
dtype: int64

In [240]:
X_test.checkin_review_count.describe()

count    1.111926e+06
mean     9.139782e+02
std      2.168039e+03
min      1.000000e+00
25%      9.400000e+01
50%      3.160000e+02
75%      8.940000e+02
max      4.010900e+04
Name: checkin_review_count, dtype: float64

In [242]:
X_test.checkin_review_count.fillna(X_test.checkin_review_count.median(),inplace = True)

In [246]:
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (4462882, 23)
X_test shape: (1115721, 23)
y_train shape: (4462882,)
y_test shape: (1115721,)


In [249]:
X_train.to_csv('X_train.csv',index = False)
y_train.to_csv('y_train.csv',index = False)

X_test.to_csv('X_test.csv',index = False)
y_test.to_csv('y_test.csv',index = False)

In [251]:
y_train.value_counts()

validated_parking
0    4345646
1     117236
Name: count, dtype: int64

Missing values handled.

In [248]:
# df_final.reset_index(drop=True,inplace=True)

# df_final.shape

(5578603, 24)

In [318]:
# df_final.to_csv('merged_final_yelp_data_new.csv',index = False)